In [2]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.gridspec import GridSpec
from tqdm.notebook import tqdm
from time import time
import seaborn as sns
import numpy as np
import pandas as pd
import os
sns.set_style('whitegrid')
plt.rc('figure', figsize=(12, 8), dpi=200)

In [3]:
import numpy.random as rg
rnormal = lambda μ, σ: int(μ + σ*rg.standard_normal())
rstudent = lambda ν, μ, σ: int(μ + σ*rg.standard_t(ν))
half_student = lambda ν, μ, σ: int(μ + σ*abs(rg.standard_t(ν)))
gamma = lambda α, β: rg.gamma(α, 1/β)
bernoulli = lambda θ: int(rg.rand() < θ)
sigmoid = lambda x: 1/(1 + np.exp(-x))

In [357]:
#indices = '132203 132325 136558 136674 136698 136704 136748 136751 136760 136800 136809 142192'.split(' ')
indices = '132203 136558 136674 136698 136704 136748 136751 136760 136800 136809 142192'.split(' ')
myindex = '136674'
sizes = range(50, 550, 50)

In [361]:
def gen_random(n):
    np.random.seed()
    P1, P2, P3, D, W = np.zeros((5, n), dtype=int)
    t = 1
    for i in range(n):
        P1[i] = rg.randint(1, 30)
        P2[i] = rg.randint(1, 30)
        P3[i] = rg.randint(1, 30)
        D[i] = np.ceil(t)
        W[i] = int(100 * rg.beta(1.2, 2.3)) + 1
        assert(W[i] > 0)
        assert(D[i] > 0)
        assert(P1[i] > 0)
        assert(P2[i] > 0)
        assert(P3[i] > 0)
        t += rg.randint(1, 20)
    A = np.vstack((P1, P2, P3, D, W))
    J = np.arange(n)
    rg.shuffle(J)
    return A.T[J].T

def score(A, J):
    T1, T2, T3 = 0, 0, 0
    P1, P2, P3, d, w = A
    N = len(J)
    C = np.zeros(N)
    D = np.zeros(N)
    for j in J:
        T1 += P1[j]
        T2 = max(T2, T1) + P2[j]
        T3 = max(T2, T3) + P3[j]
        C[j] = T3
        D[j] = max(0, C[j] - d[j])
    return np.sum(w * D) / np.sum(w)

def plot_in(A):
    P1, P2, P3, D, W = A
    P = P1, P2, P3
    N = len(D)
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_xlim(0, np.max(D))
    ax.set_ylim(0, N)
    colors = 'red', 'lightgreen', 'lightblue'
    J = np.argsort(D)
    for i, j in enumerate(J):
        d, w = D[j], W[j]
        for m in range(3):
            p, c = P[m][j], colors[m]
            ax.add_patch(patches.Rectangle((d-p, i+m*0.2), p, 0.2, color=c, alpha=w/np.max(W)))
            
def plot_out(A, J):
    P1, P2, P3, D, W = A
    P = P1, P2, P3
    N = len(D)
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_xlim(0, np.max(D))
    ax.set_ylim(0, N)
    colors = 'red', 'lightgreen', 'lightblue'
    T1, T2, T3 = 0, 0, 0
    for i, j in enumerate(J):
        d, w = D[j], W[j]
        p1, p2, p3 = P1[j], P2[j], P3[j]
        a = w / np.max(W)
        T1 += p1
        T2 = max(T1, T2) + p2
        T3 = max(T2, T3) + p3
        ax.add_patch(patches.Rectangle((T1-p1, i+0*0.2), p1, 0.2, alpha=a, color='orange'))
        ax.add_patch(patches.Rectangle((T2-p2, i+1*0.2), p2, 0.2, alpha=a, color='lightgreen'))
        ax.add_patch(patches.Rectangle((T3-p3, i+2*0.2), p3, 0.2, alpha=a, color='lightblue'))
        ax.add_patch(patches.Rectangle((d, i), 1, 1, alpha=0.5, color='red'))

def read_out(path):
    with open(path, 'r') as f: lines = f.readlines()
    L = float(lines[0].strip())
    J = np.array([int(j) for j in lines[1].strip().split(' ')])
    return L, J - 1

def read_in(path):
    with open(path, 'r') as f: lines = f.readlines()
    N = int(lines[0].strip())
    A = np.array([[int(x) for x in l.split(' ')] for l in lines[1:]]).T
    assert len(A) == 5 and len(A[0]) == N
    return A

def save_in(path, A):
    with open(path, 'w') as f:
        f.write(str(len(A[0])) + '\n')
        f.write('\n'.join(' '.join(map(str, x)) for x in A.T))

def save_out(path, A, J, debug=False):
    L = score(A, J)
    L_old, J_old = None, None
    if os.path.exists(path):
        L_old, J_old = read_out(path)
    if L_old is None or L - L_old <= -0.0001:
        L_old = 'inf' if L_old is None else f'{L_old:.4f}'
        print(f'new record! {L_old} -> {L:.4f}')
    else:
        if debug:
            print(f'no improvement {L_old:.4f} < {L:.4f}')
        return False
    with open(path, 'w') as f:
        f.write(f'{L:.4f}\n')
        f.write(' '.join(str(x) for x in J + 1))
    return True
    
def solve(A, α=1, β=1):
    P1, P2, P3, D, W = A
    T1, T2, T3 = 0, 0, 0
    P = P1, P2, P3
    N = len(D)
    J = []
    T = np.arange(N)
    
    def h(j):
        t1 = T1 + P1[j]
        t2 = max(t1, T2) + P2[j]
        t3 = max(t2, T3) + P3[j]
        C = t3
        return (C - D[j])**α * W[j]**β
    
    while len(T) > 0:
        j = max(T, key=lambda j: h(j))
        T1 += P1[j]
        T2 = max(T1, T2) + P2[j]
        T3 = max(T2, T3) + P3[j]
        J.append(j)
        T = T[T != j]
    
    return np.array(J)

def solve_all(timeit=False):
    for id in indices:
        #print(id)
        for n in sizes:
            A = read_in(f'../public/zadanie3/in/{id}/{id}_{n}.txt')
            t = time()
            J = solve(A)
            t = time() - t
            assert len(J) == n
            save_out(f'../public/zadanie3/out/136674/{id}_{n}.txt', A, J, debug=False)
            if timeit: print(f'{t*1000:.4f}')
                
def optim_all():
    for id in indices:
        print(id)
        for n in sizes:
            print(n)
            A = read_in(f'../public/zadanie3/in/{id}/{id}_{n}.txt')
            for α in [0.1, 0.2, 0.5, 1, 2, 5, 10]:
                for β in [0.1, 0.2, 0.5, 1, 2, 5, 10]:
                    J = solve(A, α, β)
                    if save_out(f'../public/zadanie3/out/136674/{id}_{n}.txt', A, J, debug=False):
                        print(f'a={α} b={β}')
        print()

def gen_random_all():
    for n in sizes:
        A = gen_random(n)
        save_in(f'../public/zadanie3/in/136674/136674_{n}.txt', A)
        
def dummy_score_all():
    for id in indices:
        for n in sizes:
            J = list(range(n))
            A = read_in(f'../public/zadanie3/in/{id}/{id}_{n}.txt')
            print(f'{score(A, J):.4f}')
        
def score_all(whose='136674'):
    for id in indices:
        for n in sizes:
            A = read_in(f'../public/zadanie3/in/{id}/{id}_{n}.txt')
            L1, J = read_out(f'../public/zadanie3/out/{whose}/{id}_{n}.txt')
            L2 = score(A, J)
            
            assert L1 - L2 <= 1e-4, f'{L1:.4f} != {L2:.4f}'
            print(f'{L2:.4f}')
        
def plot_in_out(id, n, whose='136674'):
    A = read_in(f'../public/zadanie3/in/{id}/{id}_{n}.txt')
    L, J = read_out(f'../public/zadanie3/out/{whose}/{id}_{n}.txt')
    plot_out(A, J)
    
        
#solve_all(timeit=True)
#optim_all()
    
#id = 136674
dummy_score_all()
#A = read_in(f'../public/zadanie3/in/{id}/{id}_500.txt')
#plot_in(A)
#gen_random_all()
#plot_in_out('136674', 50, whose='136674')

35.5270
62.7577
107.8933
137.3127
175.9271
218.0415
270.3993
275.0806
312.2902
349.6313
241.6829
563.6359
834.6795
1085.0589
1342.2119
1564.1602
1800.1189
1991.9175
2264.2112
2504.9950
285.7674
526.0771
639.8405
762.7996
998.2267
1315.8541
1461.4685
1621.6214
1868.0227
2009.7554
82.4087
343.0303
738.9600
1624.1531
2311.1872
3337.4547
4965.8797
5617.7165
7538.5070
8656.0682
194.1394
450.9360
580.0769
895.0821
982.0890
1197.4238
1434.6612
1602.0649
1687.2888
2017.8774
6334.7594
25137.4475
57945.1682
103498.9316
162115.4812
227313.2934
312820.2970
395065.7960
523732.8552
624294.6599
618.1900
859.4211
1471.5685
1673.4667
2228.3043
2749.3790
2980.4054
3553.7719
3871.8007
4041.2706
15757.6814
28178.1506
43134.0539
63122.6760
70788.6384
79362.4282
98253.9998
111097.1772
121446.8989
137914.2355
31.1053
59.6717
92.2028
125.1069
164.0598
209.6934
195.6263
252.4774
289.2769
278.7919
157.1279
272.8802
465.2172
611.8177
734.3041
883.9527
1007.7703
1150.9730
1308.5226
1444.8324
5300.6012
25040.1285


In [358]:
score_all()

8.4595
14.2696
21.1444
36.0401
32.5690
45.6826
58.7713
53.2586
63.4410
58.3250
137.2195
354.4217
534.6795
676.2487
848.0149
1044.3936
1183.3360
1213.1560
1402.6853
1529.7526
121.7104
182.5636
200.4132
207.5406
311.1197
422.2380
527.4688
469.6041
460.3222
583.7110
17.5130
55.4680
100.1250
337.5218
258.5217
663.3660
982.9672
625.0654
949.6061
810.2987
98.2753
176.6164
262.2526
337.3201
336.9083
439.6440
753.8911
501.9384
687.4166
845.8542
4621.9323
18506.0275
42880.9401
72694.0275
111415.1935
162188.2216
212789.8040
287006.1585
363385.8183
439920.5238
279.2760
511.3703
673.0885
990.5613
1267.6780
1403.9393
1452.2524
1802.1336
1974.8718
1975.8394
9617.8764
20378.1480
31659.3871
39632.8947
49046.7380
60121.0662
67450.7010
78665.3056
89093.6576
96427.8039
0.7481
0.0000
0.0000
0.0000
0.0000
0.0066
0.0000
0.0000
0.0000
0.0000
111.1077
209.3938
297.8452
402.9794
501.2368
587.5337
677.9411
815.6460
913.2248
1000.0369
3472.2792
15025.7219
38179.3494
64825.6527
105544.2674
153409.4726
219098.8648